<a href="https://colab.research.google.com/github/crgagne/volatility_paper_elife/blob/master/testing_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from nipype.interfaces.base import File, Directory, traits, BaseInterface, BaseInterfaceInputSpec, TraitedSpec
from nipype.utils.filemanip import split_filename
import nipype.interfaces.matlab as matlab
import os
from os import listdir
from os.path import isfile, join
import shutil

In [ ]:
class _input(BaseInterfaceInputSpec):
    in_dir = Directory(exists=True, 
                       desc='path of the directory, in which the files are associated to the Matlab function (e.g. number of these files defines how many times the Matlab function is called)', 
                       mandatory = True)
    matlab_paths= traits.List(traits.Str, desc='List of paths to add to matlabpath', mandatory= True)
    matlab_script= traits.List(traits.Str, desc='Matlab script in a list; first command corresponds to the first element of the list etc.', mandatory= True)
    #change method "Any" to the type of your input parameter;
    #for possible traits see api.py in the traits installer directory,
    #e.g. /home/aathanasiadis/.local/lib/python2.7/site-packages/traits/api.py
    input_parameter = traits.Any(desc='some input parameter',mandatory = False)


In [ ]:
class _output(TraitedSpec):
    out_file = File(exists=True,desc='filename of the out file')

In [ ]:
class matlab_template(BaseInterface):
    """ Template function which takes Matlab file(s) as input and gives file as output.
    Requires Matlab and the corresponding libraries with the filter as well as NiftiConverter
    """
    
    #defines input and output of the class
    input_spec = _input
    output_spec = _output

    #makes a list of paths of files from the directory to input; by default it takes all the files.
    def _file_selection(self, select_all_files_from_directory= True):
        src_dir= self.inputs.in_dir
        self.input_files_list = []
        if select_all_files_from_directory== False:
            def input_files_list_gen(paths_list):
                for i in paths_list:
                    self.input_files_list.append(os.path.abspath(os.path.join(src_dir,i)))
            return input_files_list_gen

        else:
            for i in listdir(src_dir):
                if isfile(join(src_dir, i)):
                    self.input_files_list.append(os.path.abspath(os.path.join(src_dir,i)))
#             self.input_files_list= list(f for f in listdir(src_dir) if isfile(join(src_dir, f)))


    def _run_interface(self,runtime):        
        mlab= matlab.MatlabCommand(mfile= False) #set to True to write matlab script file
        mlab.inputs.paths= self.inputs.matlab_paths
        
        mlab_script= '%s ' %(self.inputs.matlab_script[0])
#         for j in range(len(self.input_files_list)):     #run the matlab function as many times as the number of input files are. Needs more changes!
        for i in self.inputs.matlab_script[1:]:
            next_command = '%s '%(i)
            print('next command is '+ next_command)
            mlab_script += next_command
            print('mlab script is '+ mlab_script)
        #can also do sth with the other input_parameter
        
        # --- run the script ---
        mlab.inputs.script = """%s""" %(mlab_script)
        print( mlab.inputs.script)
        results = mlab.run()
        return results.runtime


    def _list_outputs(self):
        outputs = self._outputs().get()
        #output very dependent on the user's goal
        #in this case we have a list of input files and a list of output copied files with name realtive to the corresponding input file
        #can also create output folder in the input directory to store output files
            #for fname in self.input_files_list:
        path, base, ext = split_filename(self.input_files_list[0])
        dst_path= os.path.abspath(os.path.join(path,('new_fname_based_on_input_fname_' + base + ext)))
        dst_file= shutil.copy2(self.input_files_list[0], dst_path)
        outputs['out_file'] = dst_path
        return outputs

In [ ]:
mlab= matlab_template()

In [ ]:
mlab.inputs.in_dir = '/home/aathanasiadis/Data'

In [ ]:
mlab._file_selection(select_all_files_from_directory= False)(['scm_MP2RAGEcorr_masked0.nii', 'scm_MP2RAGEcorr_masked1.nii'])

In [ ]:
mlab.inputs.matlab_paths= ['/home/aathanasiadis/Matlab/test_functions']

In [ ]:
mlab.inputs.matlab_script= ["print_sth('hello');", "print_sth('Hello Matlab');"]

In [ ]:
o= mlab.run()

next command is print_sth('Hello Matlab'); 
mlab script is print_sth('hello'); print_sth('Hello Matlab'); 
print_sth('hello'); print_sth('Hello Matlab'); 


In [ ]:
print(o)

In [ ]:
matlab_template.help()

Template function which takes Matlab file(s) as input and gives file as output.
Requires Matlab and the corresponding libraries with the filter as well as NiftiConverter

Inputs::

        [Mandatory]
        in_dir: (a pathlike object or string representing an existing
                  directory)
                path of the directory, from which the files are associated to the
                Matlab function (e.g. number of these files defines how many times
                the Matlab function is called)
        matlab_script: (a list of items which are a unicode string)
                Matlab script in a list; first command corresponds to the first
                element of the list etc.
        matlab_paths: (a list of items which are a unicode string)
                List of paths to add to matlabpath

        [Optional]
        input_parameter: (any value)
                some input parameter

Outputs::

        out_file: (a pathlike object or string representing an existing fil